In [ ]:
%matplotlib inline
%run hammer.py

import pandas as pd
import numpy as np
import tushare as ts

code = '002398' #股票代码
body_size = 0.03 #锤子实体大小上限，基准为当日开盘价，实体不能太大，波动范围限制在3%
head_size = 0.5 #锤子上影线长度上限，基准为下影线长度，上影线要短一点，不能超过下影线的一半
tail_size = 2 #下影线与实体大小比值，下影线要大于实体两倍
length = 10 #均线移动平均日数
stoplose_trigger = 1 #当价格偏离均线满足几倍标准差时移动止损

data = ts.get_k_data(code, '2012-01-01', '2017-01-01')
data.sort_index(ascending=True, inplace=True)
data.reset_index(inplace=True) #重置index，为了让之后的循环更方便
data['pct_change'] = data['close'].pct_change()
data['ma'] = data['close'].rolling(length).mean() #移动平均
data['std'] = data['close'].rolling(length).std() #移动标准差
del data['volume']
del data['amount']
data['yes_ma'] = data['ma'].shift(1) #昨天的mean和昨天的std
data['yes_std'] = data['std'].shift(1)

data['body'] = abs(data['open'] - data['close']) #计算K线实体
data['head'] = data['high'] - data[['open', 'close']].max(axis = 1) #计算上影线，.max(axis=1)指按行取最大值
data['tail'] = data[['open', 'close']].min(axis=1) - data['low'] #计算下影线

data['body_cond'] = np.where(data['body']/data['open'] < body_size, 1, 0) #实体的大小比开盘价要小于3%，K线实体不能太大
data['head_cond'] = np.where(data['tail']==0, False, data['head'] / data['tail'] < head_size)   #上影线不能长于下影线的一半，判断尾部长度为0是为了防止除法报错
data['tail_cond'] = np.where(data['body']==0, True, (data['tail']/data['body']) > tail_size)    #下影线要比实体的两倍更长才满足条件，判断body为0表示当k线为十字星时也认为满足条件
data['hammer'] = data[['head_cond', 'body_cond', 'tail_cond']].all(axis=1) #同时满足以上三个条件才是锤子K线
data['yes_hammer'] = data['hammer'].shift(1)

#移动止损
flag = 0 #持仓记录，1代表有仓位，0代表无仓
for i in range(2*length, len(data)): #从20天开始计算，因为最初10天数据无效，10-20天数据要用来判断股票价格的下降趋势
    if flag == 1: #如果之前已持仓，判断是否止损
        stoplose_price = max(data.loc[i, 'yes_ma'] - stoplose_trigger*data.loc[i, 'yes_std'], long_open_price-long_open_delta) #计算昨天的移动止损线和固定止损线取更高的线作为止损价，移动止损线=移动平均-1倍标准差
        if data.loc[i, 'low'] < stoplose_price: #当天最低价低于止损价，则平仓止损
            flag = 0 #平仓
            data.loc[i, 'return'] = min(data.loc[i, 'open'], stoplose_price)/data.loc[i-1, 'close'] - 1 #计算平仓当天的收益，取开盘价和止损价的min是因为如果当天开盘价就小于了止损价就不一定还能涨到止损价卖出也可能一直跌，所以应该以开盘价卖出
            data.loc[i, 'trade_mark'] = -10 #表示当天平仓(用于检查的标记，开仓是10；持仓为1，平仓为-10)

        else: #如果当天最低价不低于止损价，则继续持仓
            data.loc[i, 'return'] = data.loc[i, 'close']/data.loc[i-1, 'close'] - 1 #计算当天收益率(由于之前持仓，所以收益用昨日收盘价算而不是今日开盘价算)
            data.loc[i, 'trade_mark'] = 1 #表示当天持仓(用于检查的标记)

    else: #如果之前未持仓，判断是否进行开仓
        if data.loc[i-length, 'yes_ma'] > data.loc[i, 'yes_ma']: #用十天前的移动平均和当天的移动平均判断股票价格是否为下降趋势
            if data.loc[i, 'yes_hammer']: #如果昨天下降趋势且符合锤子形态
                flag = 1 #开多仓
                long_open_price = data.loc[i, 'open'] #记录开仓时开仓价格及标准差，用于固定止损
                long_open_delta = data.loc[i, 'yes_std']
                data.loc[i, 'return'] = data.loc[i, 'close']/data.loc[i, 'open'] - 1 #计算当天收益率(假设以开盘价买入，如果是收盘价.shift(1)则假设以昨日收盘价买入)
                data.loc[i, 'trade_mark'] = 10 #表示当天开仓(用于检查的标记)

data['return'].fillna(0, inplace=True) #对之前循环中未处理的没有持仓、开仓、平仓的日期进行处理，则让这些天的return都等于0

data['strategy_return'] = (data['return'] + 1).cumprod() #累计收益
data['stock_return'] = (data['pct_change'] + 1).cumprod()

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(1,1,1)
ax.plot(data.stock_return)
ax.plot(data.strategy_return)
plt.title(code)
plt.legend()
plt.show()